In [7]:
!pip install llama-index==0.5.6
!pip install langchain==0.0.148

LlamaIndex is an open-source project that provides a simple interface between LLMs and external data sources like APIs, PDFs, SQL etc. It provides indices over structured and unstructured data, helping to abstract away the differences across data sources.

You can use it to build a vector index, a tree index, a list index, or a keyword index2. LlamaIndex gives you the ability to query your data for any downstream LLM use case, whether it’s question-answering, summarization, or a component in a chatbot.

LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain’s use-cases largely overlap with those of language models in general, including document analysis and summarization , chatbots , and code analysis1. It is designed to connect a language model to other sources of data and allow it to interact with its environment

In [8]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display
import config
os.environ["OPENAI_API_KEY"] = config.api_key

In [9]:
def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit)

    # define LLM
    # lower temperature value is more deterministic a predictable
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

In [10]:
def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("What do you want to ask? ")
        if query == "exit":
            break
        response = index.query(query)
        display(Markdown(f"### {query}\n\n<b>{response.response}</b>"))

In [11]:
construct_index("context_data/data/apple_event/")

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5609 tokens


In [12]:
ask_ai()

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2817 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 10 tokens


### cuanto cuestan los apple vision pro?

<b>
Los Apple Vision Pro cuestan 3,499 dólares.</b>